### Elasticsearch 文档创建与管理
本笔记本演示如何使用 Python 创建和管理 Elasticsearch 文档


In [ ]:
from elasticsearch_dsl import connections,Index,Document,Text,Integer,Boolean,Keyword,Date

### 连接与索引设置
建立 Elasticsearch 连接并定义文档映射结构，包含字段类型和索引配置。


In [ ]:
connection = connections.create_connection(hosts="http://localhost:9200");

In [ ]:
class Books(Document):
    title = Text()
    description = Text()
    category = Keyword()
    price = Integer()
    in_stock = Boolean()
    timestamp = Date()  # 关键字段

    class Index:
        name="books"
        settings = {
            "number_of_shards": 1,
            "number_of_replicas": 0  # 开发环境建议设为0
        }
    # class Index:
    #     name="books";

In [ ]:
if Index("books").exists():
    Index("books").delete

### 数据准备
从 CSV 文件加载书籍数据，包含标题、描述、类别、价格和库存状态等信息。


In [ ]:
import pandas as pd

df_book = pd.read_csv("./data/books.csv")
df_book

In [ ]:
#构建文档

data = df_book.iloc[0,:]
print(data)
print(df_book[df_book.title.duplicated()])  # 检查DataFrame是否有重复

### 单文档创建
演示如何创建单个文档并保存到 Elasticsearch 索引中。


In [ ]:
book = Books(
    title = data.title,description=data.description,
    category=data.category,price=data.price,in_stock=data.in_stock
)
book

### 批量文档创建
使用循环批量导入所有书籍数据到 Elasticsearch 索引。


In [ ]:
from datetime import datetime


for row_no,row in df_book.iterrows():
    print(row_no)
    book = Books(
        title = row.title,description=row.description,
        category=row.category,price=row.price,in_stock=row.in_stock,
        #timestamp=datetime.utcnow()  # 自动添加当前时间
    )
    book.save()

### 数据验证
检查数据重复性，确保导入数据的完整性。


In [ ]:
print(df_book[df_book.title.duplicated()])  # 检查DataFrame是否有重复